In [ ]:
# ==========================================
# CELL 1: ENVIRONMENT SETUP
# ==========================================
import sys
import os
import shutil
import glob
import zipfile
import torch
import pandas as pd
from PIL import Image
from tqdm.notebook import tqdm # Use notebook version for better UI
from google.colab import drive

# Install dependencies (only if missing)
try:
    import qwen_vl_utils
except ImportError:
    print("📦 Installing Libraries...")
    !pip install -q git+https://github.com/huggingface/transformers accelerate bitsandbytes qwen-vl-utils pandas pillow

# Mount Drive
if not os.path.exists('/content/drive'):
    drive.mount('/content/drive')

project_path = '/content/drive/MyDrive/AdMIRe_Project'
print("✅ Environment Ready.")

📦 Installing Libraries...
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.1/59.1 MB 14.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 521.0/521.0 kB 38.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.5/40.5 MB 20.7 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
sentence-transformers 5.1.2 requires transformers<5.0.0,>=4.41.0, but you have transformers 5.0.0.dev0 which is incompatible.
Mounted at /content/drive
✅ Environment Ready.


In [ ]:
# ==========================================
# CELL 2: DATA PREPARATION (COPY & IMPORT)
# ==========================================
import os
import glob
import zipfile
import shutil
import sys
from google.colab import drive

# 1. MOUNT DRIVE
if not os.path.exists('/content/drive'):
    drive.mount('/content/drive')

# 2. COPY THE DATASET READER
project_path = '/content/drive/MyDrive/AdMIRe_Project'
script_source = os.path.join(project_path, "admire_dataset.py")

if os.path.exists(script_source):
    shutil.copy(script_source, ".")
    print(f"✅ Copied admire_dataset.py to local runtime.")
else:
    # STRICT MODE: Stop if file is missing
    raise FileNotFoundError(f"❌ CRITICAL: 'admire_dataset.py' was not found in {project_path}. Please upload it to your Drive folder.")

# 3. IMPORT READER
from admire_dataset import AdMIReReader

# 4. PREPARE DATASETS (TRAIN & TEST)
# Ensure these match your zip names in Drive
train_zip = "AdMIRe Subtask A Train.zip"
test_zip = "AdMIRe Subtask A Test (Labelled).zip"

def unzip_if_needed(zip_name, target_dir):
    zip_path = f"{project_path}/{zip_name}"

    if not os.path.exists(zip_path):
        print(f"⚠️ Warning: {zip_name} not found in Drive. Skipping unzip.")
        return

    # Heuristic: Check if target folder is empty
    if os.path.exists(target_dir) and len(os.listdir(target_dir)) > 0:
        print(f"✅ {zip_name} seems already extracted.")
        return

    print(f"📦 Extracting {zip_name}...")
    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        zip_ref.extractall(target_dir)

unzip_if_needed(train_zip, "/content/admire_data/train_data")
unzip_if_needed(test_zip, "/content/admire_data/test_data")

# 5. INITIALIZE READERS
print("\n--- INITIALIZING READERS ---")

# Source Reader (Train) - Used for Few-Shot Examples
try:
    train_reader = AdMIReReader(data_root_path='/content/admire_data/train_data', split="Train", mode="qwen")
    print(f"✅ Source (Examples): Loaded {len(train_reader)} items.")
except Exception as e:
    print(f"❌ Error loading Train set: {e}")

# Target Reader (Test) - Used for Evaluation
try:
    try:
        test_reader = AdMIReReader(data_root_path='/content/admire_data/test_data', split="Test", mode="qwen")
    except:
        # Fallback if the TSV is named 'Dev' inside the zip
        test_reader = AdMIReReader(data_root_path='/content/admire_data/test_data', split="Dev", mode="qwen")
    print(f"✅ Target (Evaluation): Loaded {len(test_reader)} items.")
except Exception as e:
    print(f"❌ Error loading Test set: {e}")

✅ Copied admire_dataset.py to local runtime.
📦 Extracting AdMIRe Subtask A Train.zip...
📦 Extracting AdMIRe Subtask A Test (Labelled).zip...

--- INITIALIZING READERS ---
🕵️ Scanning /content/admire_data/train_data for Train TSV...
✅ Loaded TSV: /content/admire_data/train_data/train/subtask_a_train.tsv
✅ Images located at: /content/admire_data/train_data/train/
✅ Source (Examples): Loaded 70 items.
🕵️ Scanning /content/admire_data/test_data for Test TSV...
✅ Loaded TSV: /content/admire_data/test_data/test/subtask_a_test.tsv
✅ Images located at: /content/admire_data/test_data/test/
✅ Target (Evaluation): Loaded 15 items.


In [ ]:
# ==========================================
# CELL 3: MODEL DEFINITION (FULL RES / 2-SHOT)
# ==========================================
from transformers import AutoModelForVision2Seq, AutoProcessor, BitsAndBytesConfig
from qwen_vl_utils import process_vision_info
import torch
import gc

class QwenFewShot:
    def __init__(self, model_name="qwen3"):
        self.device = "cuda" if torch.cuda.is_available() else "cpu"
        self.hf_id = "Qwen/Qwen3-VL-4B-Instruct" if "qwen3" in model_name else "Qwen/Qwen2.5-VL-3B-Instruct"

        print(f"🤖 Loading {self.hf_id} (Full Resolution Mode)...")
        bnb_config = BitsAndBytesConfig(load_in_4bit=True, bnb_4bit_quant_type="nf4", bnb_4bit_compute_dtype=torch.float16)

        self.model = AutoModelForVision2Seq.from_pretrained(
            self.hf_id, quantization_config=bnb_config, device_map="auto", trust_remote_code=True
        )
        self.processor = AutoProcessor.from_pretrained(self.hf_id, trust_remote_code=True)

        # --- STANDARD RESOLUTION (Fair Comparison) ---
        self.processor.image_processor.max_pixels = 336 * 336
        self.processor.image_processor.min_pixels = 224 * 224

    def format_user_turn(self, sentence, images):
        content = []
        for i, img in enumerate(images):
            content.append({"type": "image", "image": img})
            content.append({"type": "text", "text": f"[Image {i+1}] "})

        prompt = (
            f"\nIdiom: \"{sentence}\"\n"
            "Task: Select the image that best represents the METAPHORICAL meaning.\n"
            "Steps: Define the abstract meaning, identify the matching image, and output the number.\n"
            "Final Answer Format: [[number]]"
        )
        content.append({"type": "text", "text": prompt})
        return content

    def predict_few_shot(self, target_sentence, target_images, examples):
        messages = []

        # 1. ADD EXAMPLES
        for ex in examples:
            messages.append({
                "role": "user",
                "content": self.format_user_turn(ex['text'], ex['images'])
            })
            ans_idx = ex['answer'] + 1
            ideal_response = f"The idiom \"{ex['text']}\" is abstract. Image {ans_idx} matches. Final Answer: [[{ans_idx}]]"
            messages.append({
                "role": "assistant",
                "content": [{"type": "text", "text": ideal_response}]
            })

        # 2. ADD TARGET
        messages.append({
            "role": "user",
            "content": self.format_user_turn(target_sentence, target_images)
        })

        # 3. GENERATE
        text = self.processor.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
        image_inputs, video_inputs = process_vision_info(messages)

        inputs = self.processor(text=[text], images=image_inputs, videos=video_inputs, padding=True, return_tensors="pt").to(self.model.device)

        with torch.no_grad():
            generated_ids = self.model.generate(**inputs, max_new_tokens=150)

        output = self.processor.batch_decode(generated_ids, skip_special_tokens=True)[0]
        return output.split("assistant\n")[-1].strip() if "assistant\n" in output else output

# Re-initialize and clean memory
if 'model' in globals(): del model
torch.cuda.empty_cache()
gc.collect()
model = QwenFewShot(model_name="qwen3")

🤖 Loading Qwen/Qwen3-VL-4B-Instruct (Full Resolution Mode)...


/usr/local/lib/python3.12/dist-packages/transformers/models/auto/modeling_auto.py:2276: FutureWarning: The class `AutoModelForVision2Seq` is deprecated and will be removed in v5.0. Please use `AutoModelForImageTextToText` instead.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json: 0.00B [00:00, ?B/s]

model.safetensors.index.json: 0.00B [00:00, ?B/s]

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

...

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

video_preprocessor_config.json:   0%|          | 0.00/385 [00:00<?, ?B/s]

chat_template.json: 0.00B [00:00, ?B/s]

In [ ]:
# ==========================================
# CELL 4: 2-SHOT EXECUTION (FULL RES)
# ==========================================
import time
import re
import random
import pandas as pd
from PIL import Image
from tqdm.notebook import tqdm
import torch
import gc

# Configuration
LIMIT_ITEMS = None
NUM_SHOTS = 2  # TWO-SHOT

# Helper to load images
def load_images_for_item(item_data):
    paths = item_data['image_paths']
    imgs = []
    for p in paths:
        if p != "MISSING" and os.path.exists(p):
            try: imgs.append(Image.open(p).convert("RGB"))
            except: imgs.append(Image.new('RGB', (224, 224), color='black'))
        else:
            imgs.append(Image.new('RGB', (224, 224), color='black'))
    return imgs

print(f"🚀 Starting {NUM_SHOTS}-Shot Evaluation (Full Res) on {len(test_reader)} items...")
results = []
report_data = []
save_path = f"{project_path}/results_qwen_2shot_fullres.tsv"

train_indices = list(range(len(train_reader)))

count = 0
for item in tqdm(test_reader):
    if LIMIT_ITEMS and count >= LIMIT_ITEMS: break
    start_time = time.time()

    # 1. PREPARE 2 EXAMPLES
    example_data = []
    shot_indices = random.sample(train_indices, NUM_SHOTS)

    for shot_idx in shot_indices:
        ex_item = train_reader[shot_idx]
        ex_imgs = load_images_for_item(ex_item)
        example_data.append({
            'text': ex_item['text'],
            'images': ex_imgs,
            'answer': ex_item['label']
        })

    # 2. PREPARE TARGET
    target_imgs = load_images_for_item(item)

    # 3. PREDICT
    prediction_raw = "ERROR"
    top_pred_idx = -1

    try:
        prediction_raw = model.predict_few_shot(item['text'], target_imgs, example_data)

        match = re.search(r"\[\[(\d+)\]\]", prediction_raw)
        if match:
            top_pred_idx = int(match.group(1)) - 1
        else:
            nums = re.findall(r'\d+', prediction_raw)
            top_pred_idx = int(nums[-1]) - 1 if nums else 0

    except Exception as e:
        print(f"❌ Error at item {count}: {e}")
        torch.cuda.empty_cache()

    # 4. REPORT
    is_correct = (top_pred_idx == item['label'])
    elapsed = time.time() - start_time

    report_data.append({
        "True_Answer": item['label'] + 1,
        "Model_Prediction": top_pred_idx + 1 if top_pred_idx != -1 else -1,
        "Result": "TRUE" if is_correct else "FALSE",
        "Time_Sec": round(elapsed, 2)
    })

    results.append({
        "text": item['text'],
        "prediction_raw": prediction_raw,
        "is_correct": is_correct
    })

    # 5. AGGRESSIVE CLEANUP
    del example_data, target_imgs
    torch.cuda.empty_cache()
    gc.collect()

    if count % 5 == 0:
        pd.DataFrame(results).to_csv(save_path, sep='\t', index=False)
    count += 1

# Final Report Table
df_report = pd.DataFrame(report_data)
print("\n" + "="*40)
print(f"📊 REPORT ({len(df_report)} Items)")
print("="*40)
print(df_report.to_string(index=True))
print("="*40)

acc = (df_report["Result"] == "TRUE").mean() * 100
print(f"🏆 Final {NUM_SHOTS}-Shot Accuracy: {acc:.2f}%")
pd.DataFrame(results).to_csv(save_path, sep='\t', index=False)

🚀 Starting 2-Shot Evaluation (Full Res) on 15 items...


  0%|          | 0/15 [00:00<?, ?it/s]


📊 REPORT (15 Items)
    True_Answer  Model_Prediction Result  Time_Sec
0             4                 4   TRUE     67.12
1             2                 2   TRUE     59.33
2             3                 2  FALSE     65.53
3             3                 1  FALSE     60.75
4             4                 2  FALSE     59.57
5             1                 1   TRUE     59.93
6             3                 2  FALSE     60.16
7             5                 2  FALSE     59.94
8             2                 1  FALSE     60.23
9             4                 2  FALSE     60.11
10            2                 1  FALSE     61.69
11            5                 1  FALSE     60.69
12            2                 1  FALSE     59.56
13            2                 1  FALSE     59.63
14            5                 4  FALSE     60.38
🏆 Final 2-Shot Accuracy: 20.00%
